In [105]:
import imgaug as ia
import numpy as np
from PIL import Image
import cv2 as cv
from imgaug import augmenters as iaa
import numpy as np
from random import randint, uniform, shuffle
import glob

In [106]:
def xy_to_yolo_bbox(img, augmented_bbox):
    # x_center y_center width height
    
    abs_width = augmented_bbox.x2 - augmented_bbox.x1
    abs_height = augmented_bbox.y2 - augmented_bbox.y1
    
    x_center = augmented_bbox.x1 + abs_width/2
    y_center = augmented_bbox.y1 + abs_height/2
    
    img_width = img.shape[1]
    img_height = img.shape[0]    
    
    yolo_x = x_center / img_width
    yolo_y = y_center / img_height
    
    yol_w = abs_width / img_width
    yol_h = abs_height / img_height
    
    return yolo_x, yolo_y, yol_w, yol_h


def yolo_bb_to_xy(img, txt_file):
    
    yolo_bboxes = []
    
    with open(txt_file) as f:
        for line in f:
            line = line.strip()
            yolo_bboxes.append([float(x) for x in line.split(' ')[1:]])
    
    img_width = img.shape[1]
    img_height = img.shape[0]
    
    xy_bboxes = []
    
    for bbox in yolo_bboxes:
        x_center = bbox[0] * img_width
        y_center = bbox[1] * img_height

        bbox_width = bbox[2] * img_width
        bbox_height = bbox[3] * img_height

        x1 = x_center - bbox_width/2
        y1 = y_center - bbox_height/2

        x2 = x_center + bbox_width/2
        y2 = y_center + bbox_height/2
    
        
        xy_bboxes.append(ia.BoundingBox(x1=int(x1), y1=int(y1), x2=int(x2), y2=int(y2)))
    
    return ia.BoundingBoxesOnImage(xy_bboxes, shape=img.shape)

In [107]:
images = []
bboxes = []

IMAGES_DIR = 'test_images/'

for i in glob.glob(IMAGES_DIR + '/*.jpg'):
    print(i.split('.')[0])
    file_name_no_extension = i.split('.')[0]
    
    img = cv.imread(file_name_no_extension + '.jpg')
    
    bboxes.append(yolo_bb_to_xy(img, file_name_no_extension + '.txt'))
    images.append(img)

test_images/2018-10-19-222046
test_images/1
test_images/2018-10-19-222051
test_images/2018-10-19-221933


In [138]:
def display(img):
    return Image.fromarray(img[:, :, ::-1])

def Add():
    print("add")
    val = randint(-45, 45)
    seq_l.append(iaa.Add(val))

def GaussianBlur():
    print("blur")
    val = uniform(0.0, 2.0)
    seq_l.append(iaa.GaussianBlur(val))

def Hue():
    print("hue")
    val = randint(-10,10)
    seq_l.append(iaa.AddToHueAndSaturation(val))

def Noise():
    print('noise')
    val1 = uniform(0.0, .04)
    seq_l.append(iaa.AdditiveGaussianNoise(scale=val1*225))

def Contrast():
    print('contrast')
    val1 = uniform(.2, .5)
    val2 = uniform(.5, .9)
    seq_l.append(iaa.ContrastNormalization(alpha=(val1, val2)))
    
    
def Crop():
    top = randint(0,10)
    right = randint(0,10)
    bottom = randint(0,10)
    left = randint(0,10)
    print("zoomed by: top=%d, right=%d, bottom=%d, left=%d" %(top, right, bottom, left))
    
def Zoom():
    val = uniform(1.0, 1.5)
    print("zoom by: x%f" %val)
    seq_l.append(iaa.Affine(scale=val))
    

In [146]:
seq_l = []
def augment(images, bboxs, noise=False, add=False, blur=False, hue=False, contrast=False, crop=False, zoom=False):
    if noise:
        Noise()
    if add:
        Add()
    if blur:
        GaussianBlur()
    if hue:
        Hue()
    if contrast:
        Contrast()
    if crop:
        Crop()
    if zoom:
        Zoom()
    
    seq_l.append(iaa.Fliplr(.5))
    seq = iaa.Sequential(seq_l)
    
    seq_det = seq.to_deterministic()
    
    augmented_images = seq_det.augment_images(images)
    augmented_bboxes = seq_det.augment_bounding_boxes(bboxs)
    
    print("amount of augmentations: " + str(len(seq_l)))

    return augmented_images, augmented_bboxes

In [147]:
augmented_images, augmented_bboxes = augment(images, bboxes, noise=1,add=1,blur=1, zoom=1, crop=1)

noise
add
blur
zoomed by: top=3, right=4, bottom=9, left=7
zoom by: x1.275679
amount of augmentations: 5


In [148]:
AUGMENT_DIR_NAME = 'augmented_data'

for img_num, data in enumerate(zip(augmented_images, augmented_bboxes)):
    aug_img, aug_bbox = data
    
    fp = AUGMENT_DIR_NAME + '/aug' + str(img_num) + '.jpg'
    cv.imwrite(fp, aug_img)
    
    fp = AUGMENT_DIR_NAME + '/aug' + str(img_num) + '.txt'

    with open(fp, mode='w+') as f:
        for cur_bbox in aug_bbox.bounding_boxes:
            x, y, w, h = xy_to_yolo_bbox(aug_img, cur_bbox)
            print(x,y, w, h)
            f.write('0 %f %f %f %f\n' % (x, y ,w , h))

0.8097615443686704 0.17424836341768177 0.061126292823521194 0.16536581633416333
0.6802003802318595 0.291185619253983 0.062455125276206405 0.16300344752938964
0.625053833445422 0.7305862169419027 0.1089642611201897 0.1535539723102945
0.6808647964582021 0.2805549596325011 0.07441461735037341 0.18426476677235346
0.7572726624876034 0.16361770379619983 0.07042811999231778 0.18190239796757968
0.823049868895523 0.171885994612908 0.061126292823521194 0.188989504381901
0.9266988002049718 0.22385810831793077 0.06644162263426204 0.1653658163341634
0.35330759687129015 0.4553702511857595 0.0677704550869473 0.18426476677235343
1.039649558683217 0.26992430001101914 0.07441461735037352 0.19135187318667474
0.6908310398533414 0.30417864768023867 0.08105877961379969 0.17481529155325842
0.6316979957088482 0.8026384654875025 0.10364893130944886 0.14646686589597327
